## Introduction

The dataset describes the housing market in Boston. It has 500 rows and 14 columns.  
The target variable (y) is MEDV, which is the median value of houses in thousand's.  
The features variables (x's) are:  
1) CRIM - crime rate per capita.  
2) NOX - nitric oxides concentration (parts per 10 million).  
3) DIS - weighted distances to 5 Boston business centers.  
4) RM - average number of rooms per dwelling.  
5) PTRATIO - Pupil-teacher ratio by town (categorical).  

## Data Cleaning

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from scipy import stat

ImportError: cannot import name 'stat' from 'scipy' (C:\Users\mai_t\anaconda3\lib\site-packages\scipy\__init__.py)

In [ ]:
housing = pd.read_csv('housing_data.csv', header=0)

In [ ]:
housing.head()

In [ ]:
housing.shape

In [ ]:
housing.dtypes

In [ ]:
housing['PTRATIO'] = pd.cut(housing['PTRATIO'], bins=[-float('inf'), 15, float('inf')], labels=['Low', 'High'])
housing.head()

In [ ]:
housing['PTRATIO'] = housing['PTRATIO'].astype(str)

In [ ]:
housing.dtypes

## Exploratory Analysis

In [ ]:
sns.histplot(housing['MEDV'], kde=True, color='skyblue', bins='auto', edgecolor='black')

# Title and labels
plt.title('Histogram with KDE Plot')
plt.xlabel('Values')
plt.ylabel('Frequency')

# Show the plot
plt.show()

1) The distribution of the MEDV variable does seem to be normal.  
2) The variable can be used to predict MEDV are:  
    CRIM - crime rate per capita.  
    NOX - nitric oxides concentration (parts per 10 million).  
    DIS - weighted distances to 5 Boston business centers.  
    RM - average number of rooms per dwelling.  
    PTRATIO - Pupil-teacher ratio by town (categorical). 

In [ ]:
plt.scatter(housing['CRIM'], housing['MEDV'], color='blue', alpha=0.5)

# Title and labels
plt.title('MEDV vs CRIM')
plt.xlabel('CRIM')
plt.ylabel('MEDV')

# Show the plot
plt.show()

In [ ]:
plt.scatter(housing['NOX'], housing['MEDV'], color='blue', alpha=0.5)

# Title and labels
plt.title('MEDV vs NOX')
plt.xlabel('NOX')
plt.ylabel('MEDV')

# Show the plot
plt.show()

In [ ]:
plt.scatter(housing['DIS'], housing['MEDV'], color='blue', alpha=0.5)

# Title and labels
plt.title('MEDV vs DIS')
plt.xlabel('DIS')
plt.ylabel('MEDV')

# Show the plot
plt.show()

In [ ]:
plt.scatter(housing['RM'], housing['MEDV'], color='blue', alpha=0.5)

# Title and labels
plt.title('MEDV vs RM')
plt.xlabel('RM')
plt.ylabel('MEDV')

# Show the plot
plt.show()

In [ ]:
sns.boxplot(x='PTRATIO', y='MEDV', data=housing, palette="Set2")

# Title and labels
plt.title('Box Plot: MEDV across PTRATIO')
plt.xlabel('PTRATIO')
plt.ylabel('MEDV')

# Show the plot
plt.show()

PTRATIO would be useful in predicting MEDV, since the boxplot shows that the houses in the areas with low student/teacher ratio have higher median prices, and vice versa.

In [ ]:
correlation_matrix = housing[['CRIM', 'NOX', 'DIS', 'RM']].corr()

# Create a heatmap of the correlation matrix
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', linewidths=.5)

# Title
plt.title('Correlation Plot: x variables')

Since DIS and NOX exhibit strong correlation (-0.77), I switch NOX to TAX - full-value property-tax rate per $10,000

In [ ]:
correlation_matrix = housing[['CRIM', 'TAX', 'DIS', 'RM']].corr()
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', linewidths=.5)
plt.title('Correlation Plot: x variables')

In [ ]:
correlation_matrix = housing[['MEDV','CRIM', 'TAX', 'DIS', 'RM']].corr()
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', linewidths=.5)
plt.title('Correlation Plot: x variables')

## Modelling using Linear Regression

In [ ]:
housing['PTRATIO'] = housing['PTRATIO'].map({'Low': 1, 'High': 0})
housing.head()

In [ ]:
housing.dtypes

In [ ]:
x = housing[['CRIM', 'DIS', 'TAX', 'RM', 'PTRATIO']]
y = housing['MEDV']

print(x)
print(y)

In [ ]:
X = sm.add_constant(x)

In [ ]:
# Create linear regression model

model = sm.OLS(y, X).fit()
print(model.summary())

R-square is 0.936

Coefficient interpretation:  

CRIM: an increase of one crime per capita will decrease the median house price by 142.6 dollar   
DIS: one mile increase in the average distance to the business centers will decrease the median price by 57.8 dollar  
TAX: one dollar increase in tax per $10,000 will decrease the median price by 11.2 dollar
RM: the number of room increase by 1 will increase the median house price by 7,539 dollar  
PTRATIO: if the student-teacher ratio is Low ( 15 or less), median house price will increase by 4,168 dollar

This looks like an okay model based on the R-square of 0.539, meaning the x features included in the model helped to explain 53.9% of the changes to MEDV.  
The coefficents matched the initial assumption, with CRIM, DIS, and TAX will decrease the median house price, and RM and PTRATIO wil increase the house price.

## Checking Assumptions

In [ ]:
# Calculating residuals

predictions = model.predict(X)  # X is your feature matrix
residuals = y - predictions 

In [ ]:
# Plotting residuals

sns.set(style="whitegrid")
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
stats.probplot(residuals, dist="norm", plot=plt)
plt.title('QQ-plot of Residuals')

# Histogram with kernel density estimate
plt.subplot(1, 2, 2)
sns.histplot(residuals, kde=True, bins=20)
plt.title('Histogram with KDE of Residuals')

plt.tight_layout()
plt.show()

Assuming a significance level of 0.05, the p-values from the summary table above indicate that all of the coefficients, except for DIS, are statistically significant.

The Durbin-Watson statistic of 0.769 show that the residuals are somewhat correlated. This will result in larger standard errors and reduces the accuracy of the model. The model may not perform well on new data set.

In [ ]:
fitted_values = model.predict(sm.add_constant(X))

In [ ]:
plt.figure(figsize=(8, 6))
plt.scatter(fitted_values, residuals, alpha=0.6)
plt.axhline(y=0, color='r', linestyle='--', linewidth=2)
plt.title('Residuals vs Fitted Values')
plt.xlabel('Fitted Values')
plt.ylabel('Residuals')
plt.show()

The residuals looks like they are fluctuating around zero and there is no visible pattern.  

Similar to the Durbin-Watson statistic, if there is homoskedasticity, the model will face the risk of larger standard error and reduced accuracy.

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Fit the OLS model
model = sm.OLS(y, sm.add_constant(X)).fit()

# Calculate VIF for each feature
vif_data = pd.DataFrame()
vif_data["Feature"] = X.columns
vif_data["VIF"] = [variance_inflation_factor(sm.add_constant(X).values, i) for i in range(sm.add_constant(X).shape[1])]

# Set the feature names as index
vif_data.set_index("Feature", inplace=True)

# Display the DataFrame with VIF values
print(vif_data)


The VIF of all features are all under 2, which is very low. The model does not seem to exhibit multicollinearity.

If there is high  multicollinearity, the coefficient value maybe inefficient and the sign maybe incorrect. Larger standard error will cause wider confidence interval, which caused the p-value to be unstable.